In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
with open('content.json') as content:
  data1 = json.load(content)

tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [ ]:
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(data)

                                      inputs      tags
0                                      hello  greeting
1                                   hi there  greeting
2                           nice to meet you  greeting
3   hi, is this is the pirate's organization  greeting
4                         any pirates here ?  greeting
..                                       ...       ...
81                    What is StatStream.ai?     about
82                  What does StatStream do?     about
83             What is the use of StatStream     about
84           Why should we choose StatStream     about
85                  Tell me about StatStream     about

[86 rows x 2 columns]


In [ ]:
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)

output_length = le.classes_.shape[0]
print("output length: ",output_length)

8
number of unique words :  103
output length:  9


In [ ]:

i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)



In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
3/3 [==============================] - 8s 19ms/step - loss: 2.1985 - accuracy: 0.1163
Epoch 2/200
3/3 [==============================] - 0s 18ms/step - loss: 2.1953 - accuracy: 0.1512
Epoch 3/200
3/3 [==============================] - 0s 14ms/step - loss: 2.1924 - accuracy: 0.2791
Epoch 4/200
3/3 [==============================] - 0s 23ms/step - loss: 2.1899 - accuracy: 0.3023
Epoch 5/200
3/3 [==============================] - 0s 24ms/step - loss: 2.1871 - accuracy: 0.3140
Epoch 6/200
3/3 [==============================] - 0s 38ms/step - loss: 2.1847 - accuracy: 0.3023
Epoch 7/200
3/3 [==============================] - 0s 31ms/step - loss: 2.1818 - accuracy: 0.3023
Epoch 8/200
3/3 [==============================] - 0s 15ms/step - loss: 2.1789 - accuracy: 0.3140
Epoch 9/200
3/3 [==============================] - 0s 20ms/step - loss: 2.1755 - accuracy: 0.2907
Epoch 10/200
3/3 [==============================] - 0s 22ms/step - loss: 2.1716 - accuracy: 0.2907
Epoch 11/200
3/3 [=

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)

output_length = le.classes_.shape[0]
print("output length: ",output_length)

8
number of unique words :  103
output length:  9


In [ ]:
import random
while True:
  texts_p = []
  prediction_input = input('You : ')

  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)

  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  output = model.predict(prediction_input)
  output = output.argmax()

  response_tag = le.inverse_transform([output])[0]
  print("James : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : hi
1/1 [==============================] - 0s 460ms/step
James :  Hi random person, what do'ya want ?
You : who are you
1/1 [==============================] - 0s 30ms/step
James :  James at your service
You : what is statstream
1/1 [==============================] - 0s 26ms/step
James :   We Statstream.ai revolutionlize the way businesses measure and analyze their key performance indicators (KPIs) by providing them with a powerful and intuitive platform
You : what is the use of statstream
1/1 [==============================] - 0s 25ms/step
James :  Statstream.ai empowers businesses to thrive through our innovative KPI measurement platform and cutting-edge artificial intelligence solutions
You : okay
1/1 [==============================] - 0s 22ms/step
James :  Have a nice day
